In [1]:
import pandas as pd
import numpy as np
import math
import random
import scipy
import statsmodels.formula.api as smf
import sklearn.linear_model as lm

import matplotlib.pyplot as plt
%matplotlib inline

# Data analysis

# Model Fitting

In [11]:
pdth = pd.read_csv("ph_v1_days.csv")
print(pdth.head())
print("Availible Keys: ", len(pdth.keys()))

repl_nan = lambda x: list(map(lambda xx: 0 if math.isnan(xx) else xx, x))

AVG_ph = repl_nan(pdth['PH_avg'])

xs = []
for k in pdth.keys():
    if k in ["Date", "PH_morning", "PH_midday", "PH_evening", "PH_avg"]:
        continue
    xs.append(repl_nan(pdth[k]))
    
xs=np.array(xs)
print(xs.shape)

         Date  PH_morning  PH_midday  PH_evening  PH_avg  Ailment  Gym  \
0  06.09.2017        5.41       5.25        5.55    5.40      NaN  NaN   
1  07.09.2017        7.06       5.70        5.32    6.03      NaN  1.0   
2  08.09.2017        6.26       5.75        5.43    5.81      NaN  NaN   
3  09.09.2017        5.35       6.85        6.09    6.10      NaN  NaN   
4  10.09.2017        6.62       6.34        5.23    6.06      NaN  1.0   

   Liquid  Lemon water  Tea(black/green)  ...    Cookie  Cake    Bun  \
0    1300        250.0               NaN  ...       NaN   NaN    NaN   
1    1300        250.0               NaN  ...       NaN   NaN    NaN   
2    1350        250.0               NaN  ...       NaN   NaN    NaN   
3    2350        250.0               NaN  ...       NaN   NaN    NaN   
4    1600        250.0               NaN  ...       NaN   NaN  150.0   

   Croissant  Chocolate  Candies  Halva  Marshmallow  Red caviar  Jelly  
0        NaN        NaN      NaN    NaN         